<a href="https://colab.research.google.com/github/Aman06Singh/FinalYearProject/blob/main/get_att_recc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
!unzip /content/downloads.zip

Archive:  /content/downloads.zip
   creating: downloads/6-hour_niagara_falls_canadian_side_evening_illuminations_tour/
  inflating: downloads/6-hour_niagara_falls_canadian_side_evening_illuminations_tour/1. 6-hour-niagara-falls.jpg  
   creating: downloads/best_of_niagara_falls_tour_from_niagara_falls/
  inflating: downloads/best_of_niagara_falls_tour_from_niagara_falls/1. best-of-niagara-falls-tour-from-niagara-falls-ontario-in-niagara-falls-190431.jpg  
   creating: downloads/beyond_the_bike_lanes_bike_tour/
  inflating: downloads/beyond_the_bike_lanes_bike_tour/1. beyond-the-bike-lanes.jpg  
   creating: downloads/bird_kingdom_all_day_admission_ticket/
  inflating: downloads/bird_kingdom_all_day_admission_ticket/1. bird-kingdom-all-day.jpg  
   creating: downloads/day_and_night_tour_of_niagara_falls/
  inflating: downloads/day_and_night_tour_of_niagara_falls/1. day-and-night-tour-of-niagara-falls-in-niagara-falls-193929.jpg  
   creating: downloads/epic_toronto_night_tour/
  inflati

In [13]:
pip install google_images_download

In [14]:
pip install tensorflow==1.14

In [11]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [1]:
from attractions_recc import *
from IPython.display import display
import ipywidgets as w
import pandas as pd
import re

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
att_df = pd.read_json('/content/drive/MyDrive/AMAN SINGH/Intelligent-Travel-Recommendation-System-master-1/etl/attractions.json',orient='records')

In [3]:
att_df.head()

,province,category,attraction_id,city,country,name,price,rating,avg_price,latitude,longitude
0,british_columbia,featured_tours_and_tickets,0,vancouver,canada,vancouver_city_sightseeing_tour,80.0,4.5,145.00,49.197834,-123.064995
1,british_columbia,featured_tours_and_tickets,1,vancouver,canada,vancouver_to_victoria_and_butchart_gardens_tou...,210.0,5.0,145.00,49.197834,-123.064995
2,quebec,featured_tours_and_tickets,2,montreal,canada,quebec_city_and_montmorency_falls_day_trip_fro...,115.0,4.5,115.00,45.500145,-73.572029
3,ontario,featured_tours_and_tickets,3,toronto,canada,niagara_falls_day_trip_from_toronto,169.0,5.0,212.21,43.656151,-79.384262
4,ontario,featured_tours_and_tickets,4,niagara_falls,canada,"best_of_niagara_falls_tour_from_niagara_falls,...",158.0,5.0,212.21,43.085712,-79.082428


## User details and Preferences

In [4]:
uname = w.Text(description="User Name")
place = w.Text(value = 'Province',  description="Destination")
budget = w.IntRangeSlider(min=att_df.price.min(), max=att_df.price.max(),step=10,value=[att_df.price.min(),att_df.price.max()],description="Budget")
v1 = w.VBox([uname,place, budget])
start = w.DatePicker(description='Start Date',disabled=False)
end = w.DatePicker(description='End Date',disabled=False)
v2 = w.VBox([start,end])
out = w.HBox([v1,v2])
display(out)

In [5]:
category_df = att_df.groupby('category').size().reset_index().sort_values([0],ascending=False)[:20]
categories = list(category_df.category.values)
cat_rat = dict()
def on_button_clicked(b):
    if b.description in cat_rat:
        return
    print(b.description)
    slider = w.IntSlider(min=0,max=5,step=1,description='Rate')
    display(slider)
    cat_rat[b.description] = slider
    if(len(cat_rat) < 5):
        print("Rate {x} more!\n".format(x=5-len(cat_rat)))
    
but_layout = w.Layout(width='100%', height='100px')
but_items = [w.Button(description=c, layout=but_layout) for c in categories]
on_clk = [item.on_click(on_button_clicked) for item in but_items]
c1 = w.VBox([i for i in but_items[:4]])
c2 = w.VBox([i for i in but_items[4:8]])
c3 = w.VBox([i for i in but_items[8:12]])
c4 = w.VBox([i for i in but_items[12:16]])
c5 = w.VBox([i for i in but_items[16:]])
buttons = w.HBox([c1,c2,c3,c4,c5])
print("Select and rate atleast 5 categories and rate them:")
display(buttons)

Select and rate atleast 5 categories and rate them:


private_&_custom_tours


IntSlider(value=0, description='Rate', max=5)

Rate 4 more!

food,_wine_&_nightlife


IntSlider(value=0, description='Rate', max=5)

...

IntSlider(value=0, description='Rate', max=5)

Rate 2 more!

tours_&_sightseeing


IntSlider(value=0, description='Rate', max=5)

Rate 1 more!

recommended_experiences


IntSlider(value=0, description='Rate', max=5)

In [6]:
user_name = re.sub(' ','_',uname.value.lower())
province = re.sub(' ','_',place.value.lower())
(low,high) = tuple([float(i) for i in budget.value])
begin_date = start.value
end_date = end.value
cat_rating = dict()
for key, value in cat_rat.items():
    cat_rating[key] = float(value.value)

## Model recommendation and filtering

In [7]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
filename, user, rbm_att = get_recc(att_df, cat_rating)
with_url = filter_df(filename, user, low, high, province, att_df)

Reading the data
Extracting num_rows from ratings
Preprocessing the dataset
Similar User: 46



Model restored


,attraction_id,Recommendation Score
0,0,0.018642
1,1,0.003147
2,2,0.000258
3,3,0.000034
4,4,0.000369


In [9]:
print(with_url)

                                                   name  ...                                         attraction
5     niagara_falls_in_one_day:_deluxe_sightseeing_t...  ...  https://tripadvisor.ca/AttractionProductDetail...
1310  best_of_niagara_falls_tour_from_niagara_falls,...  ...  https://tripadvisor.ca/AttractionProductDetail...
2247  niagara_falls_day_and_evening_tour_with_boat_c...  ...  https://tripadvisor.ca/AttractionProductDetail...
3315  niagara_falls_platinum_tour_from_toronto_with_...  ...  https://tripadvisor.ca/AttractionProductDetail...
1306  niagara_falls_canadian_side_tour_and_maid_of_t...  ...  https://tripadvisor.ca/AttractionProductDetail...
...                                                 ...  ...                                                ...
3239  private_tour:_niagara_falls_from_toronto_with_...  ...  https://tripadvisor.ca/AttractionProductDetail...
2800  best_of_toronto_private_food_tour_with_sightse...  ...  https://tripadvisor.ca/AttractionProductDe

In [11]:
%%capture
final = dict()
final['timeofday'] = []
final['image'] = []
final['name'] = []
final['location'] = []
final['price'] = []
final['rating'] = []
final['category'] = []

for i in range(1,(end_date - begin_date).days+2):
    for j in range(2):
        final['timeofday'].append('Morning')
    for j in range(2):
        final['timeofday'].append('Evening')

for i in range(len(final['timeofday'])): 
    if i%4 == 0: 
        final = top_recc(with_url, final)
    else:
        final = find_closest(with_url, final['location'][-1],final['timeofday'][i], final)

## Final Output

In [12]:
print(final['image'])

['downloads/niagara_falls_in_one_day:_deluxe_sightseeing_tour_of_american_and_canadian_sides/1. 6-hour-niagara-falls.jpg', 'downloads/niagara_falls_canadian_side_tour_and_maid_of_the_mist_boat_ride/1. niagara-falls-canadian.jpg', 'downloads/6-hour_niagara_falls_canadian_side_evening_illuminations_tour/1. 6-hour-niagara-falls.jpg', 'downloads/niagara_falls_night_tour_with_dinner_and_cruise/1. niagara-falls-night-tour-with-dinner-and-cruise-in-niagara-falls-218806.jpg', 'downloads/best_of_niagara_falls_tour_from_niagara_falls/1. best-of-niagara-falls-tour-from-niagara-falls-ontario-in-niagara-falls-190431.jpg', 'downloads/ultimate_toronto_tour/1. ultimate-toronto-tour.jpg', 'downloads/day_and_night_tour_of_niagara_falls/1. day-and-night-tour-of-niagara-falls-in-niagara-falls-193929.jpg', 'downloads/friday_night_wine_and_cheese_sail_in_toronto/1. friday-night-wine-and-cheese-sail-in-toronto-in-toronto-188863.jpg']


In [13]:
days = (end_date - begin_date).days + 1
display(final_output(days,final))